# Lezione 11

In [ ]:
from liblet import Grammar, Derivation, cyk2table
from L09 import cyk, to_cnf

## Simboli e input: stringhe e sequenze

Un "chiarimento" dovuto sull'uso dei termini *simboli* e *input* e sulle relative scelte implementative adottate per il software prodotto per questo corso.

In questi *handout*, i **simboli** (siano essi terminali che non) sono implementati come stringhe, che corrispondono al tipo [Text Sequence Type](https://docs.python.org/3.7/library/stdtypes.html#text-sequence-type-str) (brevemente `str`). Osservate che in Python *non esiste il tipo carattere*, pertanto anche quando i simboli sono composti da un'unica lettera, questi saranno di tipo `str`. 

Questo è evidente se, ad esempio, si ispeziona una `Grammar`.

In [ ]:
G = Grammar.from_string("""
Name -> First Last
First -> mario | franco
Last -> bruni | rossi 
""")

In [ ]:
# Sono stringhe (elementi di tipo str) i non terminali

set(G.N)

{'First', 'Last', 'Name'}

In [ ]:
# ma pure i terminali!

set(G.T)

{'bruni', 'franco', 'mario', 'rossi'}

In base alla definizione di **linguaggio**, le **parole** e le **forme sentenziali** sono sequenze di simboli (sugli opportuni alfabeti). Ragion per cui l'implementazione naturale di tali entità è fatta usando [liste](https://docs.python.org/3.7/tutorial/datastructures.html#more-on-lists) o [tuple](https://docs.python.org/3.7/tutorial/datastructures.html#tuples-and-sequences).

Di nuovo, questo è evidente se, ad esempio, si ispeziona la forma sentenziale di una `Derivation`

In [ ]:
Derivation(G).leftmost((0, 2, 4)).sentential_form()

('franco', 'rossi')

In linea di principio, quindi, l'*input* (termine informale) che, dei nostri algoritmi, è costituito da una *parola* (termine formale) deve essere una sequenza di simboli, quindi una lista o tupla.

Non ha senso pensare all'*input* come a un elemento di tipo `str`, perché questo impedisce di riconoscere (in modo non "ambiguo") i simboli di cui è composto.

Prestate attenzione alla diversità tra seguenti asserzioni:
* `('franco', 'franco')` è una parola, ma non appartiene al linguaggio generato da `G`; 
* `'francofranco'` non sono non appartiene a tale linguaggio, ma non è nemmeno una parola sull'alfabeto `G.T` dal momento che, pur essendo un oggetto di tipo `str`, non è una sequenza di simboli terminali (contenuti in `G.T`).

In [ ]:
G_cnf = to_cnf(G)

In [ ]:
# 'francofranco' conduce ad una tabella di parsing 
# col numero errato di celle (dovrebbe essere 2 x 2)

cyk2table(cyk(G_cnf, 'francofranco'))

In [ ]:
# ()'franco', 'franco') conduce ad una tabella di parsing 
# 2 x 2 che nella cella più alta non ha il simbolo di partenza
# dato che la parola non appartiene al linguaggio generato da G

cyk2table(cyk(G_cnf, ('franco', 'franco')))

In [ ]:
# ()'franco', 'rossi') conduce ad una tabella di parsing 
# 2 x 2 che conferma il riconoscimento

cyk2table(cyk(G_cnf, ('franco', 'rossi')))

### Se i simboli sono tutti lunghi uno?

Può accadere (ed è accaduto senza che lo rendessi esplicito, cosa che può avervi indotti in confusione), che se i simboli terminali sono **tutti** stringhe di lunghezza uno, l'*input* possa essere "impropriamente" implementato, invece che con una lista di stringhe di lunghezza uno, come una stringa. 

Questa "confusione" tra i tipi è resa possibile dal fatto che in Python l'espressione `INPUT[i]` è legittima sia nel caso in cui `INPUT` si riferisca a una stringa, che a una lista o a una tupla; inoltre nel caso in cui i simboli siano stringhe di lunghezza uno, le due espressiono hanno lo stesso valore.

In [ ]:
# implementazione propria

INPUT = ('p', 'i', 'p', 'p', 'o')

INPUT[1]

'i'

In [ ]:
# implementazione impropria

INPUT = 'pippo'

INPUT[1]

'i'

## Tokenizzazione e parsing

La confusione si può fare ancora più acuta quando parliamo di *tokenizzazione*, oltre che di parsing.

In tal caso ci sono due grammatiche (e quindi due linguaggi), in gioco.

La prima grammatica $G_t = (N_t, T_t, P_t, S_t)$ è quella del **tokenizzatore**, è in generale una grammatica regolare i cui terminali $T_t$ sono i caratteri dell'alfabeto di macchina (ad esempio i caratteri *Unicode*). 

Tale grammatica può essere pensata come l'unione di $k>0$ grammatiche regolari $G^k_t = (N^k_t, T_t, P^k_t, S^k_t)$ (in cui gli $N^k_t$ e $P^k_t$ sono disgiunti e gli $S^k_t$ sono distinti al variare di $k$), ciascuna delle quali riconosce un certo "tipo" di *token*. $G_t$ è usualmente definita da $N_t =  \{S_t\} \cup \bigcup N^k_t$ e $P_t =  \{S_t \to ( S^1_t | S^2_t |  \ldots | S^k_t )^* \} \cup \bigcup P^k_t$ (dove la produzione per il simbolo iniziale, qui descritta con la notazione impropria delle espressioni regolari, indica che le parole $G_t$ sono sequenze di parole delle varie $G^k_t$, ossia di token.

La seconda grammatica $G_p = (N_p, T_p, P_p, S_p)$ è quella del **parser**, è in generale una grammatica libera da contesto i cui terminali $T_p$ sono i simboli distinti delle grammatiche $G^k_t$, ossia $T_p = \{S^1_t, S^2_t, \ldots, S^k_t\}$. 

Rispetto alla situazione in cui si considera solo il parsing, in questo contesto parlare di *input* può condurre ad ancor maggior confusione: tale termine può infatti riferirsi sia alla parola (in $T_t^*$) data in input al tokenizzatore, che alla parola (in $T_p^*$) da esso restituita che sarà quindi data in input al parser.

### Un esempio

Facciamo un esempio concreto, per cercare di chiarire meglio le idee. Supponiamo di voler scrivere una calcolatrice in grado di compiere operazioni aritmetiche su interi e veriabili, come ad esempio `52* pi`.

Dapprima avremo bisogno di dividere l'espressione in token. Per farlo avremo bisogno di tre grammatiche regolari, una per gli interi: `Number -> [0-9]+`, una per le variabili `Var -> [a-z]+` e una per gli operatori `Op -> +|-|*|/` (i nostri tre tipi di token). Le metteremo assieme nella grammatica del tokenizzatore che avrà come prima produzione `S -> (' ' | Number | Var | Op)*` (osservate che c'è anche un "trucco" per buttare via gli spazi bianchi).
 
Ora avremo bisogno di una grammatica libera da contesto per le espressioni: `Expr -> Expr Op Expr | Number | Var` (il cui unico non terminale è il simbolo iniziale `Expr`).

Siamo pronti: dato l'input (`5`, `2`, `*`, ``` ```, `p`, `i`) sull'alfabeto Unicode, il tokenizzatore lo trasforma nell'input (`Number`, `Op`, `Var`) sull'alfabeto dei terminali del parser che quest'ultimo determinerà essere prodotto dalla derivazione leftmost `Expr -> Expr Op Expr | Number Op Expr -> Number Op Var`.

Ora vi domanderete, qual è il valore dell'espressione? Questa è una ulteriore "complicazione". A ciascun *token* è associato un valore, che dipende dalla porzione di stringa che deriva (sull'alfabeto $T_t$). Ad esempio, il valore del primo token è 52, il valore del secondo è "moltiplicazione" mentre il valore del terzo e ultimo è `pi`.

# <span style="color: red">Esercizi per casa</span>

* Implementare tre funzioni che, dato un automa per $L_1$ e uno per $L_2$, costruiscano rispettivamente l'automa (non deterministico) per $L_1L_2$, $L_1\cup  L_2$ e $L_1^*$.

* Implementare una funzione per la costruzione dell'*automa completo* secondo l'algoritmo presentato nella Sezione 5.5; usatelo per scrivere una funzione che costruisca l'*automa complemento* per $L^C_1 = T^* - L_1$.

* Implementate una funzione per la costruzione dell'*automa intersezione* per $L_1\cap L_2$ secondo l'algoritmo presentato nella Sezione 5.5; provate a confrontarne il comportamento di questo automa con quello per $(L^C_1 \cup L^C_2)^C$ costruito con le funzioni del punto precedente.

* Implementate una funzione per la *minimizzazione* di un DFA, o secondo l'algoritmo presentato nella Sezione 5.7, oppure secondo l'algoritmo di [Brzozowski](https://en.wikipedia.org/wiki/DFA_minimization#Brzozowski's_algorithm).